# Fine-Tuning GPT-2 com Dataset Amazon-1.3M

**Objetivo:** Treinar GPT-2 para responder perguntas sobre produtos Amazon.

**GPU:** Otimizado para NVIDIA RTX 3070 (8GB VRAM)

In [1]:
# 1️⃣ Verificar GPU
import torch

assert torch.cuda.is_available(), "❌ GPU não detectada! Este notebook requer GPU."

print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
print(f"💾 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
print(f"🔧 CUDA: {torch.version.cuda}")

✅ GPU: NVIDIA GeForce RTX 3070 Laptop GPU
💾 VRAM: 8.0 GB
🔧 CUDA: 11.8


In [2]:
# 2️⃣ Imports
import json
import math
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)

torch.cuda.empty_cache()
torch.backends.cudnn.benchmark = True
print("📦 Bibliotecas carregadas")

C:\Users\niloa\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1007 11:44:29.347000 19532 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W1007 11:44:29.347000 19532 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


📦 Bibliotecas carregadas


In [3]:
# 3️⃣ Carregar Dataset Amazon
def ler_json(caminho, max_linhas=100000):
    linhas = []
    with open(caminho, "r", encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= max_linhas:
                break
            try:
                linhas.append(json.loads(line))
            except:
                continue
    return linhas

print("📥 Carregando dataset...")
raw_data = ler_json("LF-Amazon-1.3M/trn.json", max_linhas=100000)
df = pd.DataFrame(raw_data)[["title", "content"]].dropna()

# Criar formato: Pergunta + Resposta
df["text"] = df.apply(lambda x: f"Pergunta: O que é '{x['title']}'? Resposta: {x['content']}", axis=1)
dataset = Dataset.from_pandas(df[["text"]])

print(f"✅ {len(dataset)} amostras carregadas")
print(f"\n📝 Exemplo:\n{dataset[0]['text'][:200]}...")

📥 Carregando dataset...
✅ 100000 amostras carregadas

📝 Exemplo:
Pergunta: O que é 'Girls Ballet Tutu Neon Pink'? Resposta: High quality 3 layer ballet tutu. 12 inches in length...
✅ 100000 amostras carregadas

📝 Exemplo:
Pergunta: O que é 'Girls Ballet Tutu Neon Pink'? Resposta: High quality 3 layer ballet tutu. 12 inches in length...


In [4]:
# 4️⃣ Tokenização
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize(batch):
    tokens = tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

print("✅ Tokenização concluída")

Map: 100%|██████████| 100000/100000 [00:18<00:00, 5313.94 examples/s]

✅ Tokenização concluída


In [5]:
# 5️⃣ Carregar Modelo GPT-2
model = AutoModelForCausalLM.from_pretrained("gpt2", torch_dtype=torch.float16, device_map="auto")
model.config.pad_token_id = tokenizer.eos_token_id

memory_gb = torch.cuda.memory_allocated() / 1024**3
print(f"✅ Modelo GPT-2 carregado (FP16)")
print(f"💾 Memória GPU: {memory_gb:.2f} GB")

✅ Modelo GPT-2 carregado (FP16)
💾 Memória GPU: 0.24 GB


In [6]:
# 6️⃣ Teste ANTES do Fine-Tuning
def gerar_resposta(pergunta, max_tokens=120):
    prompt = f"Pergunta: O que é '{pergunta}'? Resposta:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    with torch.no_grad(), torch.cuda.amp.autocast():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,  # Necessário para usar temperature e top_p
            pad_token_id=tokenizer.eos_token_id
        )
    
    texto = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Tentar extrair apenas a resposta, mas mostrar tudo se falhar
    if "Resposta:" in texto:
        partes = texto.split("Resposta:", 1)
        if len(partes) > 1:
            resposta = partes[1].split("Pergunta:")[0].strip()
            if resposta:
                return resposta
    
    # Se não conseguir extrair, retornar o texto completo após o prompt
    return texto[len(prompt):].strip() if len(texto) > len(prompt) else texto

print("🧪 TESTE BASELINE (sem fine-tuning)\n")
print("Q: Game of Thrones")
resposta1 = gerar_resposta('Game of Thrones')
print(f"R: {resposta1}\n")
print("Q: Smartphone Samsung Galaxy")
resposta2 = gerar_resposta('Smartphone Samsung Galaxy')
print(f"R: {resposta2}")


🧪 TESTE BASELINE (sem fine-tuning)

Q: Game of Thrones


C:\Users\niloa\AppData\Local\Temp\ipykernel_19532\2884618824.py:6: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():


R: La bajo que é 'Game of Thrones'? Resposta: O que é 'Game of Thrones'? Resposta: O que é 'Game of Thrones'? Resposta: O que é 'Game of Thrones'? Resposta: O que é 'Game of Thrones'? Resposta: O que é 'Game of Thrones'? Resposta: O que é 'Game of Thrones'? Resposta: O que é 'Game of Thrones'? Resposta: O que é 'Game of Thrones'? Resposta: O que é 'Game of Thrones'? Respost

Q: Smartphone Samsung Galaxy
R: Esta, cómo que esse cada cada. Dios, o que sólo, e a las comentarios, cada donde, donde.

Casa: Esta, o que esse cada cada. Dios, o que sólo, e a las comentarios, cada donde, donde.

Casa: Esta, o que esse cada cada. Dios, o que sólo, e a las comentarios, cada donde, donde.
R: Esta, cómo que esse cada cada. Dios, o que sólo, e a las comentarios, cada donde, donde.

Casa: Esta, o que esse cada cada. Dios, o que sólo, e a las comentarios, cada donde, donde.

Casa: Esta, o que esse cada cada. Dios, o que sólo, e a las comentarios, cada donde, donde.


In [7]:
# 7️⃣ Fine-Tuning
torch.cuda.empty_cache()

# Recarregar modelo em FP32 (mais compatível)
model = AutoModelForCausalLM.from_pretrained("gpt2").to("cuda")
model.config.pad_token_id = tokenizer.eos_token_id

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=3e-5,
    per_device_train_batch_size=10,   # Conservador e seguro
    gradient_accumulation_steps=5,    # Batch efetivo: 50
    num_train_epochs=1,
    weight_decay=0.01,
    warmup_steps=500,
    logging_steps=200,
    save_strategy="epoch",
    save_total_limit=1,
    report_to="none",
    dataloader_num_workers=0,         # Sem workers (mais estável)
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

print("🚀 Iniciando treinamento...")
print(f"📊 Batch efetivo: {10 * 5} = 50")
print(f"📊 Total de amostras: {len(tokenized_dataset)}")
print(f"📊 Total steps: ~{len(tokenized_dataset) // 50}\n")

trainer.train()

memory_used = torch.cuda.memory_allocated() / 1024**3
print(f"\n✅ Treinamento concluído!")
print(f"💾 Memória GPU usada: {memory_used:.2f} GB")

C:\Users\niloa\AppData\Local\Temp\ipykernel_19532\3694051161.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


🚀 Iniciando treinamento...
📊 Batch efetivo: 50 = 50
📊 Total de amostras: 100000
📊 Total steps: ~2000



`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
200,3.843900
400,3.353300
600,3.263700
800,3.221900
1000,3.193900
1200,3.171600
1400,3.149100
1600,3.146000
1800,3.137400
2000,3.133600



✅ Treinamento concluído!
💾 Memória GPU usada: 1.43 GB


In [8]:
# 8️⃣ Avaliar Modelo
eval_dataset = tokenized_dataset.select(range(1000))
metrics = trainer.evaluate(eval_dataset=eval_dataset)

eval_loss = metrics.get("eval_loss")
perplexity = math.exp(eval_loss) if eval_loss else None

print(f"📊 Loss: {eval_loss:.4f}")
print(f"📊 Perplexity: {perplexity:.2f}")

📊 Loss: 3.0813
📊 Perplexity: 21.79


In [9]:
# 9️⃣ Teste DEPOIS do Fine-Tuning
def gerar_resposta_finetuned(pergunta, max_tokens=150):
    """Função simplificada para gerar respostas após o fine-tuning"""
    prompt = f"Pergunta: O que é '{pergunta}'? Resposta:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.2
        )
    
    texto = tokenizer.decode(outputs[0], skip_special_tokens=True)
    resposta = texto[len(prompt):].strip() if len(texto) > len(prompt) else texto
    
    return resposta if resposta else "[Sem resposta]"

print("🎯 TESTE PÓS FINE-TUNING\n")

print("Q: Game of Thrones")
resposta1 = gerar_resposta_finetuned('Game of Thrones')
print(f"R: {resposta1}\n")
print("Q: Smartphone Samsung Galaxy")
resposta2 = gerar_resposta_finetuned('Smartphone Samsung Galaxy')
print(f"R: {resposta2}")


🎯 TESTE PÓS FINE-TUNING

Q: Game of Thrones
R: "I would say that this is the best introduction to Game's characters, from start-to&#x2019;end. It tells a story with believable plot and compelling character development."--Publishers Weekly"An excellent selection...the book will be enjoyed by all fans who love fantasy novels but are looking for an engaging read....This one should appeal equally well at home or abroad as it does in my native language," said author Joffrey Baratheon ". . ." --Library Journal on The Book Of Snow WhiteandThe Night Before Christmas"...A good reading choice if you're into action series!" -Tales From A Memory (Penguin Classics)On This Day And On That OtherDayAnd I Can Read You Now:"

Q: Smartphone Samsung Galaxy
R: "I would say that this is the best introduction to Game's characters, from start-to&#x2019;end. It tells a story with believable plot and compelling character development."--Publishers Weekly"An excellent selection...the book will be enjoyed by all f